In [ ]:
import glob
import json
import os
import numpy as np
import matplotlib.pyplot as plt
from utils import load_scan

from skimage import measure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [ ]:
MASK_FOLDER = r'C:/Users/ejrza/Downloads/mestrado/projeto/code/masters-object/POCS/preprocessing-tutorial/water-mask/*-mask.nrrd'
exams = glob.glob(MASK_FOLDER)
print(len(exams))
exams.sort()

In [ ]:
ANN_PATH = r'D:/masters_datasets/LUNA16-ann/mhd_original/dataset*'
anns = glob.glob(ANN_PATH)
print(len(anns))
anns.sort()

In [ ]:
processed = []
not_processed = []

for ann in anns[0:1]: # entire dataset with cross val 10 fold (11)
    f = open(ann)
    category = json.load(f)
    for cat in category: # training, validation
        for im in category[cat]: # every item - box, image, label (all label are 0)
            # verify if the file was already pre-processed
            current_file = im['image'].split('.mhd')[0] + '-mask.nrrd'
            if os.path.exists('water-mask/' + current_file):
                processed.append(current_file)

                ### perform nodule validation
                scan = load_scan('water-mask/' + current_file)

                ### draw the nodule mask
                axes = [scan.shape[1], scan.shape[2], scan.shape[0]]
                nodule_mask = np.zeros(axes)

                if len(im['box']) <= 1:
                    continue

                print(scan.shape)
                print(im['box'])
                print(current_file)

                for nodule in im['box']:
                    x0 = int(nodule[0])
                    x1 = int(nodule[0] + nodule[3])
                    y0 = 512 - int(nodule[1])
                    y1 = 512 - int(nodule[1] + nodule[4])
                    z0 = int(nodule[2]) * -1
                    z1 = int((nodule[2] * -1) + nodule[5])

                    print(x0, x1, y0, y1, z0, z1)
                    for i in range(x0, x1):
                        for j in range(y1, y0):
                            for k in range(z0, z1):
                                nodule_mask[i][j][k] = 1
                    print(nodule)

                p = scan.transpose(2, 1, 0)
                verts, faces, normals, values = measure.marching_cubes(p, 0)

                fig = plt.figure(figsize=(10, 10))
                ax = fig.add_subplot(111, projection='3d')

                mesh = Poly3DCollection(verts[faces], alpha=0.20)
                face_color = [0.45, 0.45, 0.75]
                mesh.set_facecolor(face_color)
                ax.add_collection3d(mesh)

                ax.set_xlim(0, p.shape[0])
                ax.set_ylim(0, p.shape[1])
                ax.set_zlim(0, p.shape[2])

                ax.voxels(nodule_mask, facecolors=[1, 0, 0, 0.8])

                # ax.view_init(20, 90)

                break

            else:
                not_processed.append(current_file)
        break
    break

    
print(len(processed))
print(len(not_processed))